In [1]:
import matplotlib.pyplot as plt
import numpy as np
import midii
import copy
from pathlib import Path

In [ ]:
print(midii.sample.dataset[0])
mid = midii.MidiFile(
    midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
)
Path(mid.filename).name

'ba_05688_-4_a_s02_m_02.mid'

In [3]:
def calculate_absolute_times(delta_times):
    """Calculates absolute times from delta times."""
    return np.cumsum(np.array(delta_times, dtype=np.int64))

In [4]:
# --- Generate Data for Plotting ---
original_abs_times = calculate_absolute_times(mid.times)

mid_quant_wo_ef = copy.deepcopy(mid)
mid_quant_wo_ef .quantize(unit="32", sync_error_mitigation=False)
quantized_abs_times_naive = calculate_absolute_times(mid_quant_wo_ef.times)

mid_quant_w_ef = copy.deepcopy(mid)
mid_quant_w_ef.quantize(unit="32")
quantized_abs_times_with_EF = calculate_absolute_times(mid_quant_w_ef.times)

# Calculate drift (deviation from original absolute time)
drift_naive = quantized_abs_times_naive - original_abs_times
drift_with_EF = quantized_abs_times_with_EF - original_abs_times

# Use event index as x-axis for simplicity
print(len(mid.times)) # 929
# event_index = np.arange(len(mid.times))
event_index = np.arange(400)

subset = slice(0, 400, 1)
# subset = slice(0, None, 1)
drift_naive = drift_naive[subset]
drift_with_EF = drift_with_EF[subset] 
event_index = event_index[subset] 

929


In [5]:
abs_time = sum(mid.times)
abs_time_quant_w_ef = sum(mid_quant_w_ef.times)
abs_time_quant_wo_ef = sum(mid_quant_wo_ef.times)
print(abs_time, abs_time_quant_w_ef, abs_time_quant_wo_ef)
print(f"Final accumulated error(w EF)={abs(abs_time - abs_time_quant_w_ef)}")
print(f"Final accumulated error(wo EF)={abs(abs_time - abs_time_quant_wo_ef)}")

32640 32640 32025
Final accumulated error(w EF)=0
Final accumulated error(wo EF)=615


In [6]:
abs_times = np.cumsum(mid.times)
abs_times_quant_w_ef = np.cumsum(mid_quant_w_ef.times)
abs_times_quant_wo_ef = np.cumsum(mid_quant_wo_ef.times)
MAE_w_ef = np.mean(abs(abs_times - abs_times_quant_w_ef))
MAE_wo_ef = np.mean(abs(abs_times - abs_times_quant_wo_ef))
print(f"Mean Absolute Error(w EF)={MAE_w_ef:.2f}")
print(f"Mean Absolute Error(wo EF)={MAE_wo_ef:.2f}")

Mean Absolute Error(w EF)=3.75
Mean Absolute Error(wo EF)=342.75


In [7]:
RMSE_w_ef = np.mean(np.square(abs_times - abs_times_quant_w_ef))
RMSE_wo_ef = np.mean(np.square(abs_times - abs_times_quant_wo_ef))
print(f"Root Mean Square Error(w EF)={RMSE_w_ef:.2f}")
print(f"Root Mean Square Error(wo EF)={RMSE_wo_ef:.2f}")

Root Mean Square Error(w EF)=18.65
Root Mean Square Error(wo EF)=154461.96
